### 1. IMPORTS

In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

In [2]:
clientes = pd.read_csv("Clientes.csv",sep=";")

### 2. DATA PREPARATION

In [3]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3407 entries, 0 to 3406
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 3407 non-null   int64  
 1   Provincia          3376 non-null   object 
 2   Nombre_y_Apellido  3361 non-null   object 
 3   Domicilio          3359 non-null   object 
 4   Telefono           3317 non-null   object 
 5   Edad               3407 non-null   int64  
 6   Localidad          3375 non-null   object 
 7   X                  3345 non-null   object 
 8   Y                  3347 non-null   object 
 9   col10              0 non-null      float64
dtypes: float64(1), int64(2), object(7)
memory usage: 266.3+ KB


In [4]:
clientes.isnull().any()

ID                   False
Provincia             True
Nombre_y_Apellido     True
Domicilio             True
Telefono              True
Edad                 False
Localidad             True
X                     True
Y                     True
col10                 True
dtype: bool

#### There are 31 null values in Provincia and 32 in Localidad, which in both cases represents 0.9% of total. 

In [5]:
clientes.isnull().sum()

ID                      0
Provincia              31
Nombre_y_Apellido      46
Domicilio              48
Telefono               90
Edad                    0
Localidad              32
X                      62
Y                      60
col10                3407
dtype: int64

In [6]:
def null_percent(table):
    return round(table.isnull().sum()/table.shape[0]*100,1)

null_percent(clientes)

ID                     0.0
Provincia              0.9
Nombre_y_Apellido      1.4
Domicilio              1.4
Telefono               2.6
Edad                   0.0
Localidad              0.9
X                      1.8
Y                      1.8
col10                100.0
dtype: float64

In [7]:
clientes.describe(include = "all")

,ID,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,X,Y,col10
count,3407.000000,3376,3361,3359,3317,3407.000000,3375,3345,3347,0.0
unique,NaN,8,3299,3357,3305,NaN,569,3315,3317,NaN
top,NaN,Buenos Aires,"Aguiar, Claudio Valentino",HOMERO E/ CAFAYATE Y CAÃ?UELAS 3870 VILLA ALB...,1,NaN,CIUDAD DE BUENOS AIRES,"-60,77135","-32,99453",NaN
freq,NaN,1860,2,2,6,NaN,458,6,6,NaN
mean,1704.000000,NaN,NaN,NaN,NaN,40.370707,NaN,NaN,NaN,NaN
std,983.660511,NaN,NaN,NaN,NaN,14.579674,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,15.000000,NaN,NaN,NaN,NaN
25%,852.500000,NaN,NaN,NaN,NaN,28.000000,NaN,NaN,NaN,NaN
50%,1704.000000,NaN,NaN,NaN,NaN,41.000000,NaN,NaN,NaN,NaN
75%,2555.500000,NaN,NaN,NaN,NaN,53.000000,NaN,NaN,NaN,NaN


In [48]:
# Removing col10 
clientes = clientes.iloc[:,:-1]

#### X, Y Coordinates need to be float so google maps search can understand them.

In [19]:
clientes.iloc[:,7] = clientes.iloc[:,7].str.replace(",",".").astype(float)
clientes.iloc[:,8] = clientes.iloc[:,8].str.replace(",",".").astype(float)

In [24]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3407 entries, 0 to 3406
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 3407 non-null   int64  
 1   Provincia          3376 non-null   object 
 2   Nombre_y_Apellido  3361 non-null   object 
 3   Domicilio          3359 non-null   object 
 4   Telefono           3317 non-null   object 
 5   Edad               3407 non-null   int64  
 6   Localidad          3375 non-null   object 
 7   X                  3345 non-null   float64
 8   Y                  3347 non-null   float64
dtypes: float64(2), int64(2), object(5)
memory usage: 239.7+ KB


#### Rows from Null values to fill

In [25]:
inlocnull = clientes[clientes.Localidad.isnull() == True].index
clientes.loc[inlocnull]

,ID,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,X,Y
11,12,NaN,EDUARDO VICENTE SARNI,CARLOS TEJEDOR 444 EL PINAR,4480-0769,52,NaN,-58.579502,-34.827881
14,15,NaN,JESUS EDUARDO SARUTTE,52 Y 235 S/N ESQUINA NEGRA,496-4503,25,NaN,-58.074614,-35.031600
19,20,NaN,ANDRES ALBERTO SCAGLIOLA,RUTA 15 Y 74 S/N LOS TALAS,462-0411,24,NaN,-57.807541,-34.898023
202,203,NaN,MERCEDES TRIGO,L. N. ALEM 387,47-7194,31,NaN,-58.381853,-34.913214
205,206,NaN,SILVANA MARIA TROTTO,GODOY CRUZ Y R. DE ESCALADA S/N BARRIO SAN MA...,47-7952,28,NaN,-58.392346,-34.931049
784,785,NaN,"Fernandez, Anibal Oscar",22 1338,451-0607,20,NaN,-57.953230,-34.937410
795,796,NaN,"Real, Victor Hugo",2 BIS Y 515 S/N,471-4086,52,NaN,-57.983100,-34.882463
804,805,NaN,"Kepa, Carlos D",RUTA 36 Y 529 S/N,491-6025,45,NaN,-58.060030,-34.977058
819,820,NaN,"Rodriguez, Ruben Osvaldo",7 (ESQ. 32) S/N,422-4898,43,NaN,-57.968474,-34.900647
848,849,NaN,REINEL ANTONIO ARIZA NIEVES,CHIRIPA Y LA QUERENCIA 623 BARRIO EL GAUCHO,4238-7458,21,NaN,-58.369575,-34.819000


In [26]:
inprovnull = clientes[clientes.Provincia.isnull() == True].index
clientes.loc[inprovnull]

,ID,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,X,Y
11,12,NaN,EDUARDO VICENTE SARNI,CARLOS TEJEDOR 444 EL PINAR,4480-0769,52,NaN,-58.579502,-34.827881
14,15,NaN,JESUS EDUARDO SARUTTE,52 Y 235 S/N ESQUINA NEGRA,496-4503,25,NaN,-58.074614,-35.031600
19,20,NaN,ANDRES ALBERTO SCAGLIOLA,RUTA 15 Y 74 S/N LOS TALAS,462-0411,24,NaN,-57.807541,-34.898023
202,203,NaN,MERCEDES TRIGO,L. N. ALEM 387,47-7194,31,NaN,-58.381853,-34.913214
205,206,NaN,SILVANA MARIA TROTTO,GODOY CRUZ Y R. DE ESCALADA S/N BARRIO SAN MA...,47-7952,28,NaN,-58.392346,-34.931049
784,785,NaN,"Fernandez, Anibal Oscar",22 1338,451-0607,20,NaN,-57.953230,-34.937410
795,796,NaN,"Real, Victor Hugo",2 BIS Y 515 S/N,471-4086,52,NaN,-57.983100,-34.882463
804,805,NaN,"Kepa, Carlos D",RUTA 36 Y 529 S/N,491-6025,45,NaN,-58.060030,-34.977058
819,820,NaN,"Rodriguez, Ruben Osvaldo",7 (ESQ. 32) S/N,422-4898,43,NaN,-57.968474,-34.900647
848,849,NaN,REINEL ANTONIO ARIZA NIEVES,CHIRIPA Y LA QUERENCIA 623 BARRIO EL GAUCHO,4238-7458,21,NaN,-58.369575,-34.819000


### 3. PROVINCIA AND LOCALIDAD WEBSCRAPPING (GOOGLE MAPS)

In [27]:
# %%capture --no-display
import warnings
warnings.filterwarnings('ignore')

chromepath = r'D:\PY\5_webscraping\Selenium\chromedriver.exe'
html = 'https://www.google.com/maps'
PAUSE_TIME = 3

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("excludeSwitches", ['enable-automation'])

browser_path = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
chrome_options.binary_location = browser_path

# RUN SILENTLY IN BACKGROUND
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(chromepath,options=chrome_options)
driver.get(html)

## COOKIES REJECT ALL BUTTON
try:
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[1]/div/div"))).click()
except:
    pass

# Reset Maps_search Column
try:
    clientes.drop("Maps_search",axis=1)
except:
    pass
clientes=clientes.assign(Maps_search= "")

print("Scanning info...")
## Complete location column
for i in range(0,clientes.shape[0]):
    ## Get Maps_search field completed only when province or localidad are nulls
    if pd.isnull(clientes.iloc[i,1]) or pd.isnull(clientes.iloc[i,6]):
        wait = WebDriverWait(driver, 10)
        findTextBox = wait.until(EC.presence_of_element_located((By.ID, "searchboxinput")))  
        findTextBox.send_keys(Keys.CONTROL + "a")
        findTextBox.send_keys(clientes.iloc[i,8],",",clientes.iloc[i,7])
        findTextBox.send_keys(Keys.ENTER)

        # driver.implicitly_wait(10)
        time.sleep(PAUSE_TIME)
        location = driver.find_element_by_xpath("/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[10]/div/div[1]")
        clientes.iloc[i,9] = location.text
    else:
        pass

driver.quit()
print("Done")

Scanning info...
Done


In [29]:
# DF Index where scrapped data is located 
list_maps_search = clientes[clientes.iloc[:,9]!=""].index.tolist()

### 4. DATA FORMAT AND FILLING

#### 4.1. Scrapped data filling

In [30]:
#Provincia fill
clientes.iloc[[x for x in list_maps_search],1] = clientes.iloc[[x for x in list_maps_search],9].str.split(",").str[-2]

In [31]:
#localidad fill
clientes.iloc[[x for x in list_maps_search],6] = clientes.iloc[[x for x in list_maps_search],9].str.split(",").str[-3]

#### 4.2. Scrapped data formatting and standardizing

In [34]:
clientes["Provincia"].unique()

array(['Buenos Aires', ' Provincia de Buenos Aires',
       'Ciudad de Buenos Aires', 'Córdoba', ' Córdoba',
       ' X5002JUI Córdoba', 'Entre Ríos', 'Mendoza', 'Neuquén',
       ' Neuquén', 'Santa Fe', ' Tucumán', 'Tucumán'], dtype=object)

In [55]:
clientes.iloc[[x for x in list_maps_search],1] = clientes.iloc[[x for x in list_maps_search],1].str.lstrip(" ")

In [56]:
clientes["Provincia"].replace("X5002JUI Córdoba","Córdoba",inplace=True)
clientes["Provincia"].replace("Buenos Aires","Provincia de Buenos Aires",inplace=True)

In [57]:
clientes["Provincia"].unique()

array(['Provincia de Buenos Aires', 'Ciudad de Buenos Aires', 'Córdoba',
       'Entre Ríos', 'Mendoza', 'Neuquén', 'Santa Fe', 'Tucumán'],
      dtype=object)

In [36]:
clientes.iloc[[x for x in list_maps_search],6].unique()

array(['Ezeiza', 'Lisandro Olmos', 'Berisso', ' B1862 Guernica',
       'Guernica', ' B1900 La Plata', ' Ringuelet', 'Abasto', ' La Plata',
       ' B1852 Burzaco', ' B1856 Glew', ' B1849 Claypole', 'Pilar Centro',
       'Belén de Escobar', 'Exaltación de la Cruz', 'Gral. Rodríguez',
       'San Fernando', 'Escobar', 'La Plata', 'Jacinto Yaben',
       'Mariano Mansilla', 'Puvr Felix Robin Ferreyra 2315', 'SMATA',
       'Colón', nan, 'Huiliches', ' T4178 Alderetes', ' T4103 AFL',
       ' Manantial', 'RN157'], dtype=object)

In [39]:
clientes.iloc[[x for x in list_maps_search],6] = clientes.iloc[[x for x in list_maps_search],6].str.lstrip(" ")

In [40]:
clientes.iloc[[x for x in list_maps_search],6].unique()

array(['Ezeiza', 'Lisandro Olmos', 'Berisso', 'B1862 Guernica',
       'Guernica', 'B1900 La Plata', 'Ringuelet', 'Abasto', 'La Plata',
       'B1852 Burzaco', 'B1856 Glew', 'B1849 Claypole', 'Pilar Centro',
       'Belén de Escobar', 'Exaltación de la Cruz', 'Gral. Rodríguez',
       'San Fernando', 'Escobar', 'Jacinto Yaben', 'Mariano Mansilla',
       'Puvr Felix Robin Ferreyra 2315', 'SMATA', 'Colón', nan,
       'Huiliches', 'T4178 Alderetes', 'T4103 AFL', 'Manantial', 'RN157'],
      dtype=object)

In [43]:
clientes["Localidad"].replace("B1862 Guernica","Guernica",inplace=True)
clientes["Localidad"].replace("B1900 La Plata","La Plata",inplace=True)
clientes["Localidad"].replace("B1852 Burzaco","Burzaco",inplace=True)
clientes["Localidad"].replace("B1856 Glew","Glew",inplace=True)
clientes["Localidad"].replace("B1849 Claypole","Claypole",inplace=True)
clientes["Localidad"].replace("Puvr Felix Robin Ferreyra 2315","Puvr Felix Robin Ferreyra",inplace=True)
clientes["Localidad"].replace("T4103 AFL","AFL",inplace=True)
clientes["Localidad"].replace("T4178 Alderetes","Alderetes",inplace=True)
clientes["Localidad"].replace(np.nan,"Neuquén",inplace=True)

In [44]:
clientes.iloc[[x for x in list_maps_search],6].unique()

array(['Ezeiza', 'Lisandro Olmos', 'Berisso', 'Guernica', 'La Plata',
       'Ringuelet', 'Abasto', 'Burzaco', 'Glew', 'Claypole',
       'Pilar Centro', 'Belén de Escobar', 'Exaltación de la Cruz',
       'Gral. Rodríguez', 'San Fernando', 'Escobar', 'Jacinto Yaben',
       'Mariano Mansilla', 'Puvr Felix Robin Ferreyra', 'SMATA', 'Colón',
       'Neuquén', 'Huiliches', 'Alderetes', 'AFL', 'Manantial', 'RN157'],
      dtype=object)

### 5. RESULTS

In [45]:
clientes.iloc[inlocnull]

,ID,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,X,Y,Maps_search
11,12,Provincia de Buenos Aires,EDUARDO VICENTE SARNI,CARLOS TEJEDOR 444 EL PINAR,4480-0769,52,Ezeiza,-58.579502,-34.827881,"Ezeiza, Provincia de Buenos Aires, Argentina"
14,15,Provincia de Buenos Aires,JESUS EDUARDO SARUTTE,52 Y 235 S/N ESQUINA NEGRA,496-4503,25,Lisandro Olmos,-58.074614,-35.031600,"Lisandro Olmos, Provincia de Buenos Aires, Arg..."
19,20,Provincia de Buenos Aires,ANDRES ALBERTO SCAGLIOLA,RUTA 15 Y 74 S/N LOS TALAS,462-0411,24,Berisso,-57.807541,-34.898023,"Berisso, Provincia de Buenos Aires, Argentina"
202,203,Provincia de Buenos Aires,MERCEDES TRIGO,L. N. ALEM 387,47-7194,31,Guernica,-58.381853,-34.913214,"Leandro N. Alem 351, B1862 Guernica, Provincia..."
205,206,Provincia de Buenos Aires,SILVANA MARIA TROTTO,GODOY CRUZ Y R. DE ESCALADA S/N BARRIO SAN MA...,47-7952,28,Guernica,-58.392346,-34.931049,"Guernica, Provincia de Buenos Aires, Argentina"
784,785,Provincia de Buenos Aires,"Fernandez, Anibal Oscar",22 1338,451-0607,20,La Plata,-57.953230,-34.937410,"Escuela N° 42, B1900 La Plata, Provincia de Bu..."
795,796,Provincia de Buenos Aires,"Real, Victor Hugo",2 BIS Y 515 S/N,471-4086,52,Ringuelet,-57.983100,-34.882463,"C. 2 Bis, C. 515 Bis &, Ringuelet, Provincia d..."
804,805,Provincia de Buenos Aires,"Kepa, Carlos D",RUTA 36 Y 529 S/N,491-6025,45,Abasto,-58.060030,-34.977058,"Abasto, Provincia de Buenos Aires, Argentina"
819,820,Provincia de Buenos Aires,"Rodriguez, Ruben Osvaldo",7 (ESQ. 32) S/N,422-4898,43,La Plata,-57.968474,-34.900647,"Escuela N°102 ""Dr. Dardo Rocha"", La Plata, Pro..."
848,849,Provincia de Buenos Aires,REINEL ANTONIO ARIZA NIEVES,CHIRIPA Y LA QUERENCIA 623 BARRIO EL GAUCHO,4238-7458,21,Burzaco,-58.369575,-34.819000,"La Querencia 623, B1852 Burzaco, Provincia de ..."


In [46]:
clientes.iloc[inprovnull]

,ID,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,X,Y,Maps_search
11,12,Provincia de Buenos Aires,EDUARDO VICENTE SARNI,CARLOS TEJEDOR 444 EL PINAR,4480-0769,52,Ezeiza,-58.579502,-34.827881,"Ezeiza, Provincia de Buenos Aires, Argentina"
14,15,Provincia de Buenos Aires,JESUS EDUARDO SARUTTE,52 Y 235 S/N ESQUINA NEGRA,496-4503,25,Lisandro Olmos,-58.074614,-35.031600,"Lisandro Olmos, Provincia de Buenos Aires, Arg..."
19,20,Provincia de Buenos Aires,ANDRES ALBERTO SCAGLIOLA,RUTA 15 Y 74 S/N LOS TALAS,462-0411,24,Berisso,-57.807541,-34.898023,"Berisso, Provincia de Buenos Aires, Argentina"
202,203,Provincia de Buenos Aires,MERCEDES TRIGO,L. N. ALEM 387,47-7194,31,Guernica,-58.381853,-34.913214,"Leandro N. Alem 351, B1862 Guernica, Provincia..."
205,206,Provincia de Buenos Aires,SILVANA MARIA TROTTO,GODOY CRUZ Y R. DE ESCALADA S/N BARRIO SAN MA...,47-7952,28,Guernica,-58.392346,-34.931049,"Guernica, Provincia de Buenos Aires, Argentina"
784,785,Provincia de Buenos Aires,"Fernandez, Anibal Oscar",22 1338,451-0607,20,La Plata,-57.953230,-34.937410,"Escuela N° 42, B1900 La Plata, Provincia de Bu..."
795,796,Provincia de Buenos Aires,"Real, Victor Hugo",2 BIS Y 515 S/N,471-4086,52,Ringuelet,-57.983100,-34.882463,"C. 2 Bis, C. 515 Bis &, Ringuelet, Provincia d..."
804,805,Provincia de Buenos Aires,"Kepa, Carlos D",RUTA 36 Y 529 S/N,491-6025,45,Abasto,-58.060030,-34.977058,"Abasto, Provincia de Buenos Aires, Argentina"
819,820,Provincia de Buenos Aires,"Rodriguez, Ruben Osvaldo",7 (ESQ. 32) S/N,422-4898,43,La Plata,-57.968474,-34.900647,"Escuela N°102 ""Dr. Dardo Rocha"", La Plata, Pro..."
848,849,Provincia de Buenos Aires,REINEL ANTONIO ARIZA NIEVES,CHIRIPA Y LA QUERENCIA 623 BARRIO EL GAUCHO,4238-7458,21,Burzaco,-58.369575,-34.819000,"La Querencia 623, B1852 Burzaco, Provincia de ..."


In [47]:
null_percent(clientes)

ID                   0.0
Provincia            0.0
Nombre_y_Apellido    1.4
Domicilio            1.4
Telefono             2.6
Edad                 0.0
Localidad            0.0
X                    1.8
Y                    1.8
Maps_search          0.0
dtype: float64